# Conv

Module for building convolutional neural networks. Includes component classes and flexible network classes for building a range of convolutional networks.

In [ ]:
#| default_exp conv

In [ ]:
#| export
from torch import nn
from fastcore import docments

In [ ]:
#| hide
from nbdev.showdoc import *

## Core

In [ ]:
#| export
class ConvNormAct(nn.Module):
    """
        Sequential block containing a convolutional layer, followed by a 
        normalisation layer and a ReLU activation. It is the basic component of
        convolutional neural networks.
    """
    def __init__(
        self,
        in_channels, # Number of channels in the input
        out_channels, # Number of channels in the output after convolution
        kernel_size=3, # Size of square kernel used for convolution (3 represents a square of 3x3 pixels)
        bias=True, # If true, a bias parameter is automatically included
        stride=2, # Size of stride
        norm=nn.BatchNorm2d, # type of normalisation applied, default BatchNorm2d. If "None", no normalisation is applied.
        act=nn.ReLU, # Activation function. If "None", no activation function is applied.
        zero_norm_weights=False
    ):
        super().__init__()
        conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, bias=bias)
        if zero_norm_weights: nn.init.constant_(conv.weight, 0.)
        self.block = nn.Sequential(
            conv,
            norm(out_channels) if norm is not None else nn.Identity(),
            act() if act is not None else nn.Identity()
        )
        
    def forward(self, x): return self.block(x)

## Resnet

In [ ]:
#| export
class ResnetStem(nn.Sequential):
    """
        First block used in a resnet, containing a series of layers 
        for downsampling the input. Input is downsampled by a factor 
        of 4, leaving a pixel grid that is 4x smaller than the input.
    """
    def __init__(
        self,
        stem_sizes: list # stem block channel sizes — [img_channels, 32, 32, 64] common
    ):
        super().__init__(
            *[
                ConvNormAct(
                    in_channels=stem_sizes[i],
                    out_channels=stem_sizes[i+1],
                    kernel_size=3,
                    stride=2 if i==0 else 1
                )
            for i in range(len(stem_sizes) - 1)
            ],
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

In [ ]:
#| export
class BottleneckBlock(nn.Module):
    """
        Main component of a resnet. Contains a series of ConvNormAct 
        layers that compute over the input, in a 'wide, narrow, wide' 
        pattern with an optional stride parameter. The input is threaded
        through the layer with a skip connection, and added to the output 
        as a residual.
    """
    def __init__(
        self,
        in_channels, # number of channels in the input
        out_channels, # number of channels in output after block
        reduction=4, # factor of reduction in the bottleneck
        stride=1 # kernel stride (only affects the first ConvNormAct layer in the block)
    ):
        super().__init__()
        reduced_features = out_channels // reduction
        
        self.block = nn.Sequential(
            ConvNormAct(in_channels, reduced_features, kernel_size=1, stride=stride), # <----- including stride enables us to stride on this layer
            ConvNormAct(reduced_features, reduced_features, kernel_size=3),
            ConvNormAct(reduced_features, out_channels, kernel_size=1)
        )
        
        self.shortcut = (
            nn.Sequential(
                ConvNormAct(in_channels, out_channels, kernel_size=1)
            ) if in_channels != out_channels else nn.Identity()
        )
        
        self.pool = (
            nn.AvgPool2d(kernel_size=3, stride=stride, padding=1) 
            if stride != 1 else nn.Identity()
        )
        
    def forward(self, x):
        residual = x
        x = self.block(x)
        residual = self.shortcut(self.pool(residual))
        x += residual
        return x

In [ ]:
#| export
class ResnetStage(nn.Sequential):
    """
        High level component enabling flexible construction of resnets 
        of different depths and sizes. Each stage contains a series of
        bottleneck blocks — the number depends on the 'depth' parameter 
        passed down from the parent class.
    """
    def __init__(
        self,
        in_channels, # Number of channels in the input
        out_channels, # Number of channels in the output
        depth, # Number of BottleneckBlocks included in the stage
        stride=2 # Stride passed down to the BottleneckBlock (only affects the first ConvNormAct layer in child BottleneckBlock)
    ):
        super().__init__(
            BottleneckBlock(in_channels, out_channels, stride=stride),
            *[
                BottleneckBlock(out_channels, out_channels, stride=stride)
                for i in range(depth - 1)
            ]
        )

In [ ]:
#| export
class ResnetNN(nn.Module):
    """
        Main resnet class that builds the network from a series of 
        subclasses. Flexible enough to enable construction of resnets 
        across a range of widths and depths. Finishes with an average 
        pool layer followed by a fully connected layer, which produces 
        the predicted classes in the output.
    """
    def __init__(
        self,
        img_channels, # Number of channels in the image
        stem_sizes, # Number of channels to use in ConvNormAct layers in the ResnetStem block — [32,32,64] is a common choice
        widths, # Widths for the output of each layer. Wider layers usually means more capabilities, but more parameters and slower training
        depths, # Number of bottleneck blocks contained in each ResnetStage
        num_classes # Number of possible labels in the training set
    ):
        super().__init__()
        stem_sizes = [img_channels, *stem_sizes]
        self.stem = ResnetStem(stem_sizes)
        
        self.stages = nn.ModuleList(
            [
                ResnetStage(stem_sizes[-1], widths[0], depths[0], stride=1),
                *[
                    ResnetStage(widths[i], widths[i+1], depths[i+1])
                    for i in range(len(widths) - 1)
                ]
            ]
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(widths[-1], num_classes)
        
    def forward(self, x):
        x = self.stem(x)
        for stage in self.stages:
            x = stage(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()